In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        filename = os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
water = pd.read_csv(filename)

In [ ]:
water = water.dropna()  

water['Solids'] = water['Solids']/100
plt.figure(figsize=(8, 8), dpi=1000)
sns.heatmap(water.corr(), annot=True,linewidth=.8, cmap="YlGnBu")

In [ ]:
print("\n数据的均衡情况：")
print(water.groupby('Potability').count())
print(water.columns)

In [ ]:
#数据均衡处理
zero = water[water['Potability']== 0]
one = water[water['Potability']== 1]
from sklearn.utils import resample
df_minority_upsampled = resample(one, replace=True, n_samples = 1200, random_state=123)
#concatenate
water = pd.concat([zero, df_minority_upsampled])
from sklearn.utils import shuffle
water = shuffle(water)

#将数据划分为训练集、测试集 
water_data = water.iloc[:,:9]
water_target = water.iloc[:,9]
water_data_train, water_data_test, water_target_train, water_target_test = \
train_test_split(water_data, water_target, test_size=0.2)
#print(water_data_train.shape, water_data_test.shape, water_target_train.shape, water_target_test.shape)

#数据标准化
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
stdScaler = StandardScaler().fit(water_data_train)
water_trainStd = stdScaler.transform(water_data_train)
water_testStd = stdScaler.transform(water_data_test)
water_Std = stdScaler.transform(water_data)

In [ ]:
df =  pd.DataFrame(np.zeros(56).reshape(7,8),index=['accuracy', 'precision', 'recall', 'f1_score', 'cohen_kappa_score',
       'cross_val_score mean', 'cross_val_score std'],columns=['SVM', 'LR', 'KNN', 'GNB', 'DT', 'RFC', 'ETC', 'GBC'])
df.index.name = "Model"
df

In [ ]:
def ModelEvaluation(model,n):
    global df
    p_model = model.predict(water_testStd)
    from sklearn.metrics import accuracy_score
    print(str(model)[:-2] + '训练集准确率：', accuracy_score(water_target_train, model.predict(water_trainStd)))
    print(str(model)[:-2] + '测试集准确率：', accuracy_score(water_target_test, p_model))
    df.iloc[0,n] = accuracy_score(water_target_test, p_model)

    from sklearn.metrics import precision_score
    print(str(model)[:-2] + '训练集精确率：', precision_score(water_target_train, model.predict(water_trainStd)))
    print(str(model)[:-2] + '测试集精确率：', precision_score(water_target_test, p_model))
    df.iloc[1,n] = precision_score(water_target_test, p_model)

    from sklearn.metrics import recall_score, f1_score, cohen_kappa_score
    print("召回率：", recall_score(water_target_test, p_model))
    print("F1值：", f1_score(water_target_test, p_model))
    print("cohen_kappa_score：", cohen_kappa_score(water_target_test, p_model))

    df.iloc[2,n] = recall_score(water_target_test,p_model)
    df.iloc[3,n] = f1_score(water_target_test,p_model)
    df.iloc[4,n] = cohen_kappa_score(water_target_test,p_model)

    #评估模型预测结果
    from sklearn.metrics import classification_report
    print("分类报告：\n", classification_report(water_target_test,p_model))

    scores = cross_val_score(model, water_Std, water_target, cv=10)
    print("%0.4f accuracy with a standard deviation of %0.4f" % (scores.mean(), scores.std()))
    df.iloc[5,n]  = scores.mean()
    df.iloc[6,n]  = scores.std()
    df = np.round(df,4)

In [ ]:
def ModelBaggingEvaluation(model,n):
    global dfBagging
    p_model = model.predict(water_testStd)
    from sklearn.metrics import accuracy_score
    print(str(model)[:-2] + '训练集准确率：', accuracy_score(water_target_train, model.predict(water_trainStd)))
    print(str(model)[:-2] + '测试集准确率：', accuracy_score(water_target_test, p_model))
    dfBagging.iloc[0,n] = accuracy_score(water_target_test, p_model)

    from sklearn.metrics import precision_score
    print(str(model)[:-2] + '训练集精确率：', precision_score(water_target_train, model.predict(water_trainStd)))
    print(str(model)[:-2] + '测试集精确率：', precision_score(water_target_test, p_model))
    dfBagging.iloc[1,n] = precision_score(water_target_test, p_model)

    from sklearn.metrics import recall_score, f1_score, cohen_kappa_score
    print("召回率：", recall_score(water_target_test, p_model))
    print("F1值：", f1_score(water_target_test, p_model))
    print("cohen_kappa_score：", cohen_kappa_score(water_target_test, p_model))

    dfBagging.iloc[2,n] = recall_score(water_target_test,p_model)
    dfBagging.iloc[3,n] = f1_score(water_target_test,p_model)
    dfBagging.iloc[4,n] = cohen_kappa_score(water_target_test,p_model)

    #评估模型预测结果
    from sklearn.metrics import classification_report
    print("分类报告：\n", classification_report(water_target_test,p_model))

    scores = cross_val_score(model, water_Std, water_target, cv=10)
    print("%0.4f accuracy with a standard deviation of %0.4f" % (scores.mean(), scores.std()))
    dfBagging.iloc[5,n]  = scores.mean()
    dfBagging.iloc[6,n]  = scores.std()
    dfBagging = np.round(dfBagging,4)

In [ ]:
dfBagging =  pd.DataFrame(np.zeros(56).reshape(7,8),index=['accuracy', 'precision', 'recall', 'f1_score', 'cohen_kappa_score',
       'cross_val_score mean', 'cross_val_score std'],columns=['SVM', 'LR', 'KNN', 'GNB', 'DT', 'RFC', 'ETC', 'GBC'])
dfBagging.index.name = "BaggingModel"
dfBagging

# 支持向量机分类

In [ ]:
#建立SVM模型
from sklearn.svm import SVC

svm = SVC().fit(water_trainStd, water_target_train)   #在训练集上建立SVC分类模型
#SVC(kernel="linear", C=0.025)
#SVC(gamma=2, C=1)
print("建立SVM模型为:\n", svm)
ModelEvaluation(model=svm,n=0)
print(df)

# 逻辑斯蒂分类

In [ ]:
#建立预测模型
from sklearn.linear_model import LogisticRegression
log = LogisticRegression().fit(water_trainStd, water_target_train)
print("建立LogisticRegression模型为:\n", log)

ModelEvaluation(model=log,n=1)
print(df)

# K最近邻分类

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV 
# 网格搜索和交叉验证\随机搜索和交叉验证

knn = KNeighborsClassifier()
knn.fit(water_trainStd, water_target_train) #训练模型

ModelEvaluation(model=knn,n=2)
print(df)

# 高斯朴素贝叶斯分类

In [ ]:
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB().fit(water_trainStd, water_target_train)
print("建立GaussianNB模型为:\n", gnb)
ModelEvaluation(model=gnb,n=3)
print(df)

# 分类决策树

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(water_trainStd, water_target_train)

print("使用建立DecisionTreeClassifier模型为:\n", dtc)
ModelEvaluation(dtc,4)
print(df)

# 随机森林分类

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(water_trainStd, water_target_train)
print("建立RandomForestClassifier模型为:\n", rfc)
ModelEvaluation(rfc,5)
print(df)

# 极限随机森林 Extra Trees Classifier

In [ ]:
#极限随机森林 Extra Trees Classifier
from sklearn.model_selection import GridSearchCV #网格搜索
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier()
etc.fit(water_trainStd, water_target_train)
ModelEvaluation(etc,6)
print(df)

# 梯队提升分类树

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier().fit(water_trainStd, water_target_train)
print("建立GradientBoostingClassifier模型为:\n", gbc)
ModelEvaluation(gbc,7)

print(df)

 ETC is great!